<a href="https://colab.research.google.com/github/OscarBedford/MLCourse_Weekly_Exercises/blob/main/Exercise_13_4_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task 4- Same as task 3, but using a higher-level Gaussian for the mu-parameter (not the
variance component) as a hyper-prior for the lower-level region slopes. 500 MCMC
iterations is enough. Report output of PyMC3.summary(credible_interval=0.90).

In [ ]:
%%capture
!pip install nilearn
!pip install pymc3

In [ ]:
%%capture
import numpy as np
from nilearn import datasets
from nilearn.input_data import NiftiLabelsMasker
from nilearn.image import index_img
import nibabel as nib

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pymc3 as pm
import scipy.stats as stats
import arviz as az
from pymc3 import Bernoulli, Model, HalfCauchy
from pymc3 import invlogit, sample, summary
from numpy import mean
from numpy import std
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.cross_decomposition import PLSRegression as PLSR
from sklearn.cross_decomposition import CCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression as LR

In [ ]:
%%capture
brain_data = datasets.fetch_oasis_vbm(n_subjects=100)
yeo = datasets.fetch_atlas_schaefer_2018(n_rois=100) # this needs to be modified for some tasks
masker = NiftiLabelsMasker(labels_img=yeo.maps, standardize=True, memory='nilearn_cache')
input_variables = masker.fit_transform(brain_data.gray_matter_maps)
output_variable = np.array(brain_data.ext_vars.mf == 'F', dtype=int) # gives 1 for females and 0 for males

In [ ]:
# We will define the "age" hyperprior inside the model, like in Exercise 3
with pm.Model() as model:
      # We first extract ages from brain_data object
      ages = brain_data.ext_vars.age
      # Now we median-split the sample in order to find the value that divides people into low and high age
      age_median = np.median(ages)

      # We now define a binary hyperprior based on age, and we call it "age"
      age = pm.Bernoulli('age', p=0.5, shape=len(ages))
      low = age[ages < age_median]
      high = age[ages >= age_median]

      # We define a Gaussian hyperprior for the mu-parameter of the lower-level region slopes
      hyperprior = pm.Normal('hyperprior', mu=0, sd=1, shape=100)

      # We define the intercept term as Gaussian with prior (mu=0, sd=1)
      intercept = pm.Normal('intercept', mu=0, sd=1)

      # We define the bottom-level region slopes (or betas) as a normal distribution with priors mu=hyperprior (gaussian) and sd=1
      beta_coefficients = pm.Normal('beta_coefficients', mu=hyperprior, sd=1, observed=input_variables)

      # We calculate the probabilities using the invlogit function, as specified in the instructions
      probabilities = pm.invlogit(intercept + input_variables @ beta_coefficients + age)

       # We define likelihood as a normal distribution with mean equal to probabilities and sd=1
      likelihood = pm.Normal('likelihood', mu=probabilities, sd=1, observed=output_variable)

      # We sample from the posterior distribution using 500 MCMCs draws
      trace = pm.sample(500, return_inferencedata=True)

      # We're ready to summarize the results of the MCMC sampling
      stats = pm.summary(trace, hdi_prob=0.9)

In [ ]:
# Let's take a look at the summary table
from google.colab import data_table
data_table.DataTable(stats, include_index=True, num_rows_per_page=201)

,mean,sd,hdi_5%,hdi_95%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
age[0],0.645,0.479,0.000,1.000,0.013,0.009,1332.0,1000.0,1.00
age[1],0.650,0.477,0.000,1.000,0.013,0.009,1360.0,1000.0,1.00
age[2],0.656,0.475,0.000,1.000,0.012,0.009,1459.0,1000.0,1.00
age[3],0.157,0.364,0.000,1.000,0.013,0.009,847.0,847.0,1.00
age[4],0.160,0.367,0.000,1.000,0.011,0.008,1030.0,1000.0,1.00
...,...,...,...,...,...,...,...,...,...
hyperprior[96],0.000,0.099,-0.151,0.177,0.002,0.004,2094.0,587.0,1.00
hyperprior[97],0.001,0.102,-0.158,0.171,0.002,0.005,2349.0,591.0,1.00
hyperprior[98],-0.002,0.095,-0.148,0.166,0.002,0.004,1982.0,676.0,1.00
hyperprior[99],-0.004,0.099,-0.184,0.144,0.002,0.004,2557.0,571.0,1.01
